In [38]:
from openeo.internal.graph_building import PGNode
from openeo.rest.datacube import DataCube
from openeo.processes import array_create
import openeo

# Load collection with spatial extent and band selection
# URL is dynamically injected from the selected scene
graph = PGNode(
    "load_collection",
    id="sentinel-2-l2a",
    spatial_extent={
        "east": {"from_parameter": "spatial_extent_east"},
        "north": {"from_parameter": "spatial_extent_north"},
        "south": {"from_parameter": "spatial_extent_south"},
        "west": {"from_parameter": "spatial_extent_west"},
        "crs": "EPSG:3857",
    },
    temporal_extent=["2025-07-01", "2025-07-31"],
    bands=[
        "reflectance|b02",
        "reflectance|b03",
        "reflectance|b04",
    ],
    properties={
        "eo:cloud_cover": {
            "process_graph": {
                "cc": {
                    "process_id": "lt",
                    "arguments": {"x": {"from_parameter": "value"}, "y": 20},
                    "result": True,
                }
            }
        }
    },
)

datacube = DataCube(graph=graph)

reduced = datacube.process(
    "apply_pixel_selection",
    pixel_selection="first",
    data=datacube,
)

# Extract bands for use in algorithms
B02 = reduced.band("reflectance|b02")  # Blue
B03 = reduced.band("reflectance|b03")  # Green
B04 = reduced.band("reflectance|b04")  # Red

In [ ]:
def viz(data):
    """Apply true color transformation to spectral bands."""
    B02, B03, B04 = (
        data[0],
        data[1],
        data[2],
    )
    # True color for land (enhanced)
    true_color_b = B04 * 3
    true_color_r = B03 * 3
    true_color_g = B02 * 5
    return array_create([true_color_r, true_color_g, true_color_b])


# Apply the visualization function
map_viz = reduced.apply_dimension(dimension="bands", process=viz)

# Scale values to 0-255 range for PNG output
map_viz = map_viz.linear_scale_range(
    input_min=0, input_max=1, output_min=0, output_max=255
)

# Save as PNG and return JSON representation
map_viz = map_viz.save_result("PNG")
print(map_viz.to_json())

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "bands": [
          "reflectance|b02",
          "reflectance|b03",
          "reflectance|b04"
        ],
        "id": "sentinel-2-l2a",
        "properties": {
          "eo:cloud_cover": {
            "process_graph": {
              "cc": {
                "process_id": "lt",
                "arguments": {
                  "x": {
                    "from_parameter": "value"
                  },
                  "y": 20
                },
                "result": true
              }
            }
          }
        },
        "spatial_extent": {
          "east": {
            "from_parameter": "spatial_extent_east"
          },
          "north": {
            "from_parameter": "spatial_extent_north"
          },
          "south": {
            "from_parameter": "spatial_extent_south"
          },
          "west": {
            "from_parameter": "spatial_ext

In [40]:
# Bretagne
spatial_extent_west = -5.0
spatial_extent_south = 40
spatial_extent_east = 5.0
spatial_extent_north = 50

connection = openeo.connect(
    "https://api.explorer.eopf.copernicus.eu/openeo"
).authenticate_oidc_authorization_code()
# connection = openeo.connect(
#     "http://localhost:8081"
# ).authenticate_oidc_authorization_code()

# Create XYZ Service
service_extent = [
    spatial_extent_west,
    spatial_extent_south,
    spatial_extent_east,
    spatial_extent_north,
]

service = connection.create_service(
    map_viz,
    type="XYZ",
    title="Sentinel-2 L2A RGB - July 2025 - Bretagne",
    description="Sentinel-2 L2A RGB composite for July 2025 over Bretagne",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": service_extent,
    },
)
print(service)

Preflight process graph validation failed: [400] InvalidRequest: 1 validation error:
  {'type': 'missing', 'loc': ('body', 'id'), 'msg': 'Field required', 'input': {'process_graph': {'loadcollection1': {'process_id': 'load_collection', 'arguments': {'bands': ['reflectance|b02', 'reflectance|b03', 'reflectance|b04'], 'id': 'sentinel-2-l2a', 'properties': {'eo:cloud_cover': {'process_graph': {'cc': {'process_id': 'lt', 'arguments': {'x': {'from_parameter': 'value'}, 'y': 20}, 'result': True}}}}, 'spatial_extent': {'east': {'from_parameter': 'spatial_extent_east'}, 'north': {'from_parameter': 'spatial_extent_north'}, 'south': {'from_parameter': 'spatial_extent_south'}, 'west': {'from_parameter': 'spatial_extent_west'}, 'crs': 'EPSG:3857'}, 'temporal_extent': ['2025-07-01', '2025-07-31']}}, 'applydimension1': {'process_id': 'apply_dimension', 'arguments': {'data': {'from_node': 'loadcollection1'}, 'dimension': 'bands', 'process': {'process_graph': {'arrayelement1': {'process_id': 'array_el

<Service service_id='b40907a6-8c96-490d-8bac-66b80256ce67'>
